In [1]:
import graphviz
import json
import sqlite3
import sys, os, argparse
import math
import time
import collections

In [2]:
def run_sql_query(db, query):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(query)
    result_data = json.dumps(cur.fetchall())
    result_json = json.loads(result_data)
    return result_json

In [28]:
def gen_records(graph, nodes, edge_dict_list):
    graph.attr('node', shape='record', fontname='Courier', size='6,6')
    graph.node_attr['fontname'] = "Courier"
    graph.node_attr['color'] = "lightgreen"
    graph.node_attr['fillcolor'] = "lightblue"
    graph.node_attr['style'] = 'filled'
    graph.edge_attr['fontname'] = "Courier"
    graph.graph_attr['rankdir'] = "RL"
    graph.node_attr['fontsize'] = "10"

    for node in nodes:
        graph.node(node.get("id"), 
                   node.get("txt"), 
                   fillcolor=node.get("fillcolor"),
                   rank=node.get("rank"))

    edges = []
    for edge_dict in edge_dict_list:
        graph.edge(
            edge_dict.get("src"),
            edge_dict.get("dest"),
            label=edge_dict.get("label"),
            penwidth=edge_dict.get("penwidth")
        )

def gen_node(node_id, node_txt, fillcolor, rank):
    node = {}
    node["id"] = node_id
    node["txt"] = node_txt
    node["fillcolor"] = fillcolor
    node["rank"] = rank
    return node


In [29]:
def _total_num_of_caps(db):
    count_caps_q = "SELECT COUNT(*) FROM cap_info"
    return run_sql_query(db, count_caps_q)


In [30]:
def show_caps_between_two_libs(db, graph, lib1, lib2):
    lib1_caps_q = "SELECT * FROM cap_info WHERE cap_loc_path LIKE '%" + str(lib1) + "%'"
    lib1_caps = run_sql_query(db, lib1_caps_q)

    lib2_caps_q = "SELECT * FROM cap_info WHERE cap_loc_path LIKE '%" + str(lib2) + "%'"
    lib2_caps = run_sql_query(db, lib2_caps_q)
                    
    lib1_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + str(lib1) + "%'"
    lib1_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + str(lib1) + "%'"
    lib1_start_addrs = run_sql_query(db, lib1_start_q)
    lib1_end_addrs = run_sql_query(db, lib1_end_q)

    lib2_start_q = "SELECT start_addr FROM vm WHERE mmap_path LIKE '%" + str(lib2) + "%'"
    lib2_end_q = "SELECT end_addr FROM vm WHERE mmap_path LIKE '%" + str(lib2) + "%'"
    lib2_start_addrs = run_sql_query(db, lib2_start_q)
    lib2_end_addrs = run_sql_query(db, lib2_end_q)

    # node showing the loaded libraries, with cap records pointing into it
    # We have all the information here at this point, just need to graph it

    # First we create the two library nodes
    nodes = []
    edges = []

    nodes.append(gen_node(lib1, lib1, "lightblue", "same"))
    nodes.append(gen_node(lib2, lib2, "pink", "same"))

    for cap1 in lib1_caps:
        cap_loc_addr = cap1[0]
        cap_path = cap1[1]
        cap_addr = cap1[2]
        cap_perms = cap1[3]
        cap_base = cap1[4]
        cap_top = cap1[5]

        penwidth_weight = 1;
        
        for lib2_addr_index in range(len(lib2_start_addrs)):
            lib2_start_addr = lib2_start_addrs[lib2_addr_index][0]
            lib2_end_addr = lib2_end_addrs[lib2_addr_index][0]

            # Test if cap1's pointer address is in lib2's address range
            if cap_addr >= lib2_start_addr and cap_addr <= lib2_end_addr:
                print("cap1 to lib2: " + cap_perms)
                for props in edges:
                    if props.get("src") == lib1 and props.get("dest") == lib2:
                        penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                        edges.remove(props)
                        break
                edges.append({"src":lib1, "dest":lib2, "label":cap_perms, "penwidth":str(penwidth_weight)})

    for cap2 in lib2_caps:
        cap_loc_addr = cap2[0]
        cap_path = cap2[1]
        cap_addr = cap2[2]
        cap_perms = cap2[3]
        cap_base = cap2[4]
        cap_top = cap2[5]

        for lib1_addr_index in range(len(lib1_start_addrs)):
            lib1_start_addr = lib1_start_addrs[lib1_addr_index][0]
            lib1_end_addr = lib1_end_addrs[lib1_addr_index][0]

            # Test if cap2's pointer address is in lib1's address range
            if cap_addr >= lib1_start_addr and cap_addr <= lib1_end_addr:
                print("cap2 to lib1: " + cap_perms)

                for props in edges:
                    if props.get("src") == lib2 and props.get("dest") == lib1 and props.get("label") == cap_perms:
                        penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                        edges.remove(props)
                        break
                edges.append({"src":lib2, "dest":lib1, "label":cap_perms, "penwidth":str(penwidth_weight)})

    gen_records(graph, nodes, edges)

In [49]:
def show_caps_between_two_comparts(db, graph, c1, c2):
    
    # Get all the cap_addr (out) from the source compartment (for both c1 and c2)
    dest_caps_from_src1_q = "SELECT cap_addr, perms FROM cap_info INNER JOIN vm ON cap_info.cap_loc_path = vm.mmap_path where vm.compart_id="+c1;
    dest_caps_from_src1_json = run_sql_query(db, dest_caps_from_src1_q)

    dest_caps_from_src2_q = "SELECT cap_addr, perms FROM cap_info INNER JOIN vm ON cap_info.cap_loc_path = vm.mmap_path where vm.compart_id="+c2;
    dest_caps_from_src2_json = run_sql_query(db, dest_caps_from_src2_q)
    
    # Get the start and end addr to get the addr range to compart cap_addr with, to find out if any caps from
    # c1 are pointing to c2 and vice versa
    addr_ranges_c1_as_dest_q = "SELECT start_addr, end_addr FROM vm WHERE compart_id=" + c1
    addr_ranges_c1_as_dest_json = run_sql_query(db, addr_ranges_c1_as_dest_q)

    addr_ranges_c2_as_dest_q = "SELECT start_addr, end_addr FROM vm WHERE compart_id=" + c2
    addr_ranges_c2_as_dest_json = run_sql_query(db, addr_ranges_c2_as_dest_q)

    # We have all the information here at this point, just need to graph it

    # First we create the two comparts nodes
    nodes = []
    edges = []
    
    # Get the paths for each compartment node
    c1_paths_q = "SELECT distinct mmap_path FROM vm WHERE compart_id=" + c1
    c1_paths_json = run_sql_query(db, c1_paths_q)
    
    c2_paths_q = "SELECT distinct mmap_path FROM vm WHERE compart_id=" + c2
    c2_paths_json = run_sql_query(db, c2_paths_q)

    path1 = ""
    count = 0
    for c1_path in c1_paths_json:
        path1 += os.path.basename(c1_path[0])
        if (count != len(c1_paths_json)-1):
            path1 += ", "
        count += 1    
    path2 = ""
    count = 0
    for c2_path in c2_paths_json:
        path2 += os.path.basename(c2_path[0])
        if (count != len(c2_paths_json)-1):
            path2 += ", "
        count += 1  
        
    print(path1)
    print(path2)
    nodes.append(gen_node(c1, c1+":"+path1, "lightblue", "same"))
    nodes.append(gen_node(c2, c2+":"+path2, "pink", "same"))
    
    # Iterate through each cap_addr from source c1 and find out if any of them point to c2
    for addr_ranges_c2_as_dest in addr_ranges_c2_as_dest_json:
        penwidth_weight = 1;
        
        start_addr = addr_ranges_c2_as_dest[0]
        end_addr = addr_ranges_c2_as_dest[1]
        
        print("src:"+c1+" dest:"+c2+" dest addr range:"+start_addr+"-"+end_addr)
        
        for dest_cap_from_src1 in dest_caps_from_src1_json:
            dest_cap = dest_cap_from_src1[0]
            dest_perms = dest_cap_from_src1[1]
            
            # Now check if any of the dest caps are within the range of the dest address range
            if dest_cap >= start_addr and dest_cap <= end_addr:
                for props in edges:
                    if (props.get("src") == c1 and props.get("dest") == c2):
                        penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                        edges.remove(props)
                        break
                edges.append({"src":c1, "dest":c2, "label":dest_perms, "penwidth":str(penwidth_weight)})
    
    # Iterate through each cap_addr from source c2 and find out if any of them point to c1
    for addr_ranges_c1_as_dest in addr_ranges_c1_as_dest_json:
        penwidth_weight = 1;
        
        start_addr = addr_ranges_c1_as_dest[0]
        end_addr = addr_ranges_c1_as_dest[1]
                
        for dest_cap_from_src2 in dest_caps_from_src2_json:
            dest_cap = dest_cap_from_src2[0]
            dest_perms = dest_cap_from_src2[1]
            
            # Now check if any of the dest caps are within the range of the dest address range
            if dest_cap >= start_addr and dest_cap <= end_addr:
                for props in edges:
                    if (props.get("src") == c2 and props.get("dest") == c1):
                        penwidth_weight = math.log((10**(float(props.get("penwidth"))) + 1), 10)
                        edges.remove(props)
                        break
                edges.append({"src":c2, "dest":c1, "label":dest_perms, "penwidth":str(penwidth_weight)})

    gen_records(graph, nodes, edges)

In [59]:
CONFIG_FILE = 'chericat_cli.config'

def main(argv):
    dbpath="../../chericat_dbs/"
    dbname="konsole.sql"
    
    if (os.path.isfile(CONFIG_FILE)):
        with open(CONFIG_FILE) as f:
            sys.argv = f.read().split(',')
    else:
        sys.argv = ['chericat_cli', '-d', dbpath+dbname, '-cc','20', '6']

    parser = argparse.ArgumentParser(prog='chericat_cli')
    parser.add_argument(
        '-d', 
        help='The database to use for the queries', 
        required=True,
    )
    
    parser.add_argument(
        '-r',
        help='Executes the SQL query on the provided db',
        nargs=1,
    )

    parser.add_argument(
        '-cl',
        help="Show capabilities between two loaded libraries <libname 1> <libname 2>",
        nargs=2,
    )

    parser.add_argument(
        '-cc',
        help="Show capabilities between two loaded compartments <compart 1> <compart 2>",
        nargs=2,
    )

    args = parser.parse_args()

    if args.d:
        db = args.d

    if args.r:
        print(run_sql_query(db, args.r[0]))

    if args.cl:
        start = time.perf_counter()
        digraph = graphviz.Digraph('G', filename=args.cl[0]+'_vs_'+args.cl[1]+'.gv')
        show_caps_between_two_libs(db, digraph, args.cl[0], args.cl[1])
        end = time.perf_counter()
        print("Capabilities between two libraries, graph generated in " + str(end-start) + "s")
        digraph.render(directory='graph-output', view=True)

    if args.cc:
        start = time.perf_counter()
        digraph = graphviz.Digraph('G', filename=dbname+'.cc_graph.gv')
        show_caps_between_two_comparts(db, digraph, args.cc[0], args.cc[1])
        end = time.perf_counter()
        print("Capabilities between two compartments, graph generated in " + str(end-start) + "s")
        digraph.render(directory='graph-output', view=True)
    
    return None

if __name__ == '__main__':
    main(sys.argv)


libKF5GlobalAccel.so.5.108.0, libKF5GlobalAccel.so.5.108.0(.plt), libKF5GlobalAccel.so.5.108.0(.got), Stack
libKF5NewStuffWidgets.so.5.108.0, libKF5NewStuffWidgets.so.5.108.0(.plt), libKF5NewStuffWidgets.so.5.108.0(.got)
src:20 dest:6 dest addr range:0x404cc000-0x404d2000
src:20 dest:6 dest addr range:0x404d2000-0x404e1000
src:20 dest:6 dest addr range:0x404e1000-0x404e5000
src:20 dest:6 dest addr range:0x404e5000-0x404f4000
src:20 dest:6 dest addr range:0x404f4000-0x404f5000
src:20 dest:6 dest addr range:0x404f5000-0x40504000
src:20 dest:6 dest addr range:0x40504000-0x40506000
Capabilities between two compartments, graph generated in 0.974567165998451s
